In [1]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import netron
import pathlib
import numpy as np

In [2]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# Define the model architecture
# Define the model architecture.
# model = tf.keras.Sequential([
#   tf.keras.layers.InputLayer(input_shape=(28, 28)),
#   tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
#   tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
#   tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(10)
# ])

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(20, activation='relu', input_shape=(28, 28)),
  tf.keras.layers.Dense(10)
])


# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_data=(test_images, test_labels)
)

Epoch 1/5


ValueError: in user code:

    File "d:\dp\dp\code\openvino_notebooks_py37\venv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "d:\dp\dp\code\openvino_notebooks_py37\venv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\dp\dp\code\openvino_notebooks_py37\venv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "d:\dp\dp\code\openvino_notebooks_py37\venv\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\dp\dp\code\openvino_notebooks_py37\venv\lib\site-packages\keras\engine\training.py", line 1053, in compute_loss
        y, y_pred, sample_weight, regularization_losses=self.losses
    File "d:\dp\dp\code\openvino_notebooks_py37\venv\

# Convert the model to the TensorFlow Lite format with full integer quantization

In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [input_value]

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to uint8
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

# Save the model to disk

In [ ]:
tflite_models_dir = pathlib.Path("models")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

model_path = r"models/mnist_tflite_model_quant"

# Save the model:
tflite_model_file = tflite_models_dir/"mnist_tflite_model_quant.tflite"
if not tflite_model_file.is_file():
    tflite_model_file.write_bytes(tflite_model_quant)
    print("Model saved to: ", tflite_model_file)

# Model details to .txt file

In [ ]:
if not pathlib.Path(model_path + r'_details.txt').is_file():

    interpreter_saved_details = tf.lite.Interpreter(model_path=model_path + r'.tflite')
    interpreter_saved_details.allocate_tensors()

    with open(model_path + r'_details.txt', 'w') as f:
        f.write('Tensor details:\n\n')
        for dict in interpreter_saved_details.get_tensor_details():
            i = dict['index']
            tensor_name = dict['name']
            scales = dict['quantization_parameters']['scales']
            zero_points = dict['quantization_parameters']['zero_points']
            tensor = interpreter_saved_details.tensor(i)()
            type = dict['dtype']

            tensor_arr = np.array(tensor)

            f.write(f'{i} {type} {tensor_name} \n scales:\n {scales} \n zero_points:\n {zero_points} \n tensor_shape:\n {tensor.shape}\n tensor:\n {np.array2string(tensor_arr, threshold=np.inf,max_line_width=np.inf, separator=", ")}\n')
            f.write('\n\n------------------------------------------------------------------------------------------------------------------------\n\n')

        for item in interpreter_saved_details.get_tensor_details():
            f.write(str(item).replace('{\'name','\n{\'name'))

    print('Details saved to: ', model_path + r'_details.txt')

# Evaluate the TF Lite model

In [ ]:
interpreter = tf.lite.Interpreter(model_path=model_path + r'.tflite')

input_type = interpreter.get_input_details()[0]
print('input: ', input_type['dtype'], input_type['index'])
output_type = interpreter.get_output_details()[0]
print('output: ', output_type['dtype'], output_type['index'])

In [ ]:
def evaluate_model(interpreter):
  interpreter.allocate_tensors()
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]
  input_details = interpreter.get_input_details()[0]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    # Pre-processing: add batch dimension and convert to match with the model's input data format.

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details["dtype"] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest probability.
    output = interpreter.get_tensor(output_index)
    digit = np.argmax(output[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [ ]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
  global test_images

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_images[test_image_index]
    test_label = test_labels[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions


In [ ]:
print('Quant TFLite test_accuracy (evaluate_model):', evaluate_model(interpreter))

test_image_indices = range(10000)
predictions = run_tflite_model(tflite_model_file, test_image_indices)
print('Quant TFLite test_accuracy (run_tflite_model):', np.sum(predictions == test_labels[test_image_indices]) * 1.0 / len(test_image_indices))
# print('Prediction results: \n', predictions)

# Netron

In [ ]:
netron.start(model_path + r'.tflite')

# Change the value of the tensor

In [ ]:
print(interpreter.get_tensor(2))

In [ ]:
arr = np.zeros((20, 784), dtype='int8')
interpreter.set_tensor(2, arr)

In [ ]:
print(interpreter.get_tensor(2))

In [ ]:
# arr[0][0] = 4
# interpreter.set_tensor(2, arr)
# interpreter.tensor(2)()

In [ ]:
# tensor_details = interpreter.get_tensor_details()
# for tensor in tensor_details:
#   if tensor['name'] == 'sequential/dense/MatMul':
#     weight_shape = tensor['shape']
#     weight_idx = tensor['index']
#     weight = interpreter.get_tensor(weight_idx)
#     weight = np.zeros(weight_shape,dtype='int8')
#     print(weight)
#     interpreter.set_tensor(weight_idx, weight)

In [ ]:
print('Quant TFLite test_accuracy (evaluate_model):', evaluate_model(interpreter))

test_image_indices = range(10000)
predictions = run_tflite_model(tflite_model_file, test_image_indices)
print('Quant TFLite test_accuracy (run_tflite_model):', np.sum(predictions == test_labels[test_image_indices]) * 1.0 / len(test_image_indices))
# print('Prediction results: \n', predictions)